In [4]:
!nvidia-smi


Sun Jan 25 14:19:26 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   40C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
%%writefile task1_sum.cu

#include <iostream>              // Ввод и вывод в консоль
#include <vector>                // Контейнер std::vector
#include <chrono>                // Измерение времени на CPU
#include <cuda_runtime.h>        // CUDA Runtime API

// Размер массива
#define N 100000

// Количество потоков в одном блоке CUDA
#define THREADS_PER_BLOCK 256

// Макрос для проверки ошибок CUDA
// Если произошла ошибка, выводится сообщение и программа завершается
#define CUDA_CHECK(err) \
    if (err != cudaSuccess) { \
        std::cerr << "CUDA error: " << cudaGetErrorString(err) \
                  << " (code " << err << ")" << std::endl; \
        exit(1); \
    }

// CUDA-ядро для параллельной редукции
// data         - входной массив
// partialSums  - массив частичных сумм (по одной на блок)
// n            - количество элементов во входном массиве
__global__ void reduceKernel(const float* data, float* partialSums, int n) {

    // Shared memory для хранения данных одного блока
    // Каждый поток загружает один элемент
    __shared__ float sdata[THREADS_PER_BLOCK];

    // Локальный индекс потока внутри блока
    int tid = threadIdx.x;

    // Глобальный индекс элемента массива
    int idx = blockIdx.x * blockDim.x + tid;

    // Загрузка данных из глобальной памяти в shared memory
    // Если индекс выходит за пределы массива, записываем 0
    sdata[tid] = (idx < n) ? data[idx] : 0.0f;

    // Синхронизация всех потоков блока
    __syncthreads();

    // Параллельная редукция внутри блока
    // На каждом шаге количество активных потоков уменьшается в 2 раза
    for (int s = blockDim.x / 2; s > 0; s >>= 1) {
        if (tid < s) {
            sdata[tid] += sdata[tid + s];
        }
        // Синхронизация перед следующим шагом
        __syncthreads();
    }

    // Первый поток блока записывает сумму блока в глобальную память
    if (tid == 0) {
        partialSums[blockIdx.x] = sdata[0];
    }
}

// Последовательное суммирование массива на CPU
float cpuSum(const std::vector<float>& data) {
    float sum = 0.0f;
    for (float x : data) {
        sum += x;
    }
    return sum;
}

int main() {

    // Инициализация входного массива на CPU
    // Все элементы равны 1.0
    std::vector<float> h_data(N, 1.0f);

    // ---------------- CPU ----------------

    // Засекаем время начала вычислений на CPU
    auto cpu_start = std::chrono::high_resolution_clock::now();

    // Вычисление суммы на CPU
    float cpu_result = cpuSum(h_data);

    // Засекаем время окончания вычислений
    auto cpu_end = std::chrono::high_resolution_clock::now();

    // Вычисляем время выполнения в миллисекундах
    std::chrono::duration<double, std::milli> cpu_time = cpu_end - cpu_start;

    // ---------------- GPU ----------------

    // Указатели на память GPU
    float *d_data, *d_partial;

    // Количество блоков CUDA
    int blocks = (N + THREADS_PER_BLOCK - 1) / THREADS_PER_BLOCK;

    // Выделение памяти на GPU под входной массив
    CUDA_CHECK(cudaMalloc(&d_data, N * sizeof(float)));

    // Выделение памяти под частичные суммы блоков
    CUDA_CHECK(cudaMalloc(&d_partial, blocks * sizeof(float)));

    // Копирование входных данных с CPU на GPU
    CUDA_CHECK(cudaMemcpy(d_data,
                           h_data.data(),
                           N * sizeof(float),
                           cudaMemcpyHostToDevice));

    // CUDA-события для измерения времени выполнения ядра
    cudaEvent_t start, stop;
    CUDA_CHECK(cudaEventCreate(&start));
    CUDA_CHECK(cudaEventCreate(&stop));

    // Запись времени начала
    CUDA_CHECK(cudaEventRecord(start));

    // Запуск CUDA-ядра
    reduceKernel<<<blocks, THREADS_PER_BLOCK>>>(d_data, d_partial, N);

    // Проверка ошибок запуска ядра
    CUDA_CHECK(cudaGetLastError());

    // Ожидание завершения всех потоков GPU
    CUDA_CHECK(cudaDeviceSynchronize());

    // Запись времени окончания
    CUDA_CHECK(cudaEventRecord(stop));
    CUDA_CHECK(cudaEventSynchronize(stop));

    // Вычисление времени выполнения ядра в миллисекундах
    float gpu_time = 0.0f;
    CUDA_CHECK(cudaEventElapsedTime(&gpu_time, start, stop));

    // Копирование частичных сумм с GPU на CPU
    std::vector<float> h_partial(blocks);
    CUDA_CHECK(cudaMemcpy(h_partial.data(),
                           d_partial,
                           blocks * sizeof(float),
                           cudaMemcpyDeviceToHost));

    // Финальное суммирование частичных результатов на CPU
    float gpu_result = cpuSum(h_partial);

    // Вывод результатов
    std::cout << "CPU sum: " << cpu_result << std::endl;
    std::cout << "GPU sum: " << gpu_result << std::endl;
    std::cout << "CPU time (ms): " << cpu_time.count() << std::endl;
    std::cout << "GPU kernel time (ms): " << gpu_time << std::endl;

    // Освобождение памяти GPU
    cudaFree(d_data);
    cudaFree(d_partial);

    return 0;
}


Overwriting task1_sum.cu


In [5]:
!nvcc -arch=sm_75 task1_sum.cu -o task1_sum


In [6]:
!./task1_sum

CPU sum: 100000
GPU sum: 100000
CPU time (ms): 2.1038
GPU kernel time (ms): 0.087072
